In [1]:
import pandas as pd
import os

# Read in Reviews

In [2]:
review_json_path = '../data/yelp_academic_dataset_review.json'

In [3]:
size = 500_000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [4]:
type(review)

pandas.io.json._json.JsonReader

In [5]:
#pd.DataFrame(review)

In [6]:
review

In [7]:
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_merged = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    #chunk_merged = pd.merge(business_RV, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True)

ValueError: Expected object or value

In [10]:
df['date'].count()

8021122

In [8]:
df['date'].min()

'2004-10-12 10:13:32'

# Business

In [3]:
business = pd.read_json('../data/yelp_academic_dataset_business.json', lines=True)

In [54]:
# open_bus = business[business['is_open'] ==1]
# LV = open_dec[open_dec['city'] == 'Las Vegas'].copy()
# LV.reset_index(drop=True, inplace=True)
# LV.dropna(subset=['categories'], inplace=True)
# LV_rest = LV[LV['categories'].str.contains('Restaurant')]
# LV_rest.reset_index(drop=True, inplace=True)
# LV_rest_small = LV_rest[(LV_rest['review_count'] > 300) & (LV_rest['review_count'] < 1000)]

In [6]:
# LV_rest_small.head(2)

In [ ]:
# LV_rest_small['review_count']

In [ ]:
# LV_rest_small.to_csv('../data/LasVegas_starter.csv')

In [5]:
# business.head(2)

In [7]:
open_bus = business[business['is_open'] == 1]
closed_bus = business[business['is_open'] == 0]

In [8]:
LV_open = open_bus[open_bus['city'] == 'Las Vegas'].copy()
LV_closed = closed_bus[closed_bus['city'] == 'Las Vegas'].copy()

# Reviews + Businesses

In [2]:
LV_sample = pd.read_csv('../data/LV_400.csv')

In [3]:
def business_reviews(review_json, business_json, city_name):
    chunk_list = []
    for chunk_review in review:
        chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
        chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
        chunk_merged = pd.merge(LV_sample, chunk_review, on='business_id', how='left')
        print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
        chunk_list.append(chunk_merged)
    df_reviews = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
    
    #uncomment if you want csv of business in given city
    df_reviews.to_csv(f'../data/{city_name}_reviews.csv', index = False)
    return df_reviews

# https://towardsdatascience.com/converting-yelp-dataset-to-csv-using-pandas-2a4c8f03bd88

In [4]:
# import Review.json
review_json_path = '../data/yelp_academic_dataset_review.json'
size = 500_000
rows = 1_000_000

review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [5]:
# function that joins city business information with yelp reviews
LV_reviews = business_reviews(review, LV_sample, 'Las Vegas')

6812 out of 500,000 related reviews
5610 out of 500,000 related reviews
5653 out of 500,000 related reviews
3949 out of 500,000 related reviews
4385 out of 500,000 related reviews
5028 out of 500,000 related reviews
4739 out of 500,000 related reviews
4910 out of 500,000 related reviews
5672 out of 500,000 related reviews
4534 out of 500,000 related reviews
4616 out of 500,000 related reviews
4794 out of 500,000 related reviews
4834 out of 500,000 related reviews
5104 out of 500,000 related reviews
4181 out of 500,000 related reviews
4331 out of 500,000 related reviews
468 out of 500,000 related reviews


In [16]:
len(LV_sample['business_id'].unique())

400

In [17]:
len(LV_reviews['business_id'].unique())

400

In [20]:
LV_reviews.dropna(inplace=True)

In [21]:
len(LV_reviews['business_id'].unique())

400

In [23]:
LV_reviews['is_open'].value_counts()

1    36734
0    35641
Name: is_open, dtype: int64